## Lab 1 - Recommendation system smallest 

In [163]:
import pandas as pd 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import plotly_express as px
import seaborn as sns 
import matplotlib.pyplot as plt

In [110]:
movies="C:/Users/Noah7/code/Github/AI-Noah-Eliasson-OPA23/data/ml-latest-small/ml-latest-small/movies.csv"
ratings="C:/Users/Noah7/code/Github/AI-Noah-Eliasson-OPA23/data/ml-latest-small/ml-latest-small/ratings.csv"

df_movies=pd.read_csv(movies, usecols=["movieId", "title"], dtype={"movieId": "int32", "title": "str"})
df_ratings=pd.read_csv(ratings, usecols=["userId", "movieId", "rating"], dtype={"userId": "int32", "movieId": "int32", "rating": "float32"})

In [111]:
# spare matrix 
#        Users
#        [4,4,5]
# movies [4,3,2]
#        [3,1,3]

# think of it as database tabels, we can also pivot tables, using one tables index to use that as index on another table, to then get new info about the certain index in this 
# case movieId 

movies_users= df_ratings.pivot(index="movieId", columns="userId", values="rating").fillna(0)
mat_movies_users= csr_matrix(movies_users.values)

mat_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float32'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [112]:
# Eculidean distance 
# Manhattan distance
# Minkowski distance
# Cosine similarity  
model_KNN= NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=20)

In [113]:
model_KNN.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [114]:
# Recommender(movie_name) => List of Movies recommended 

def recommender(movie_name, data, model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies["title"])[2]
    print("Movies Selected: ", df_movies["title"][idx], "Index: ", idx)
    print("Searching for recommendations.....")
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies["title"][i].where(i!=idx))
        

recommender("iron man 2", mat_movies_users, model_KNN, 20)

Movies Selected:  Iron Man 2 (2010) Index:  7324
Searching for recommendations.....
9656                     The Greatest Showman (2017)
8590                           Jurassic World (2015)
7324                                             NaN
8946    Kung Fu Panda: Secrets of the Masters (2011)
9605           The Nut Job 2: Nutty by Nature (2017)
7936                          Treasure Island (2012)
9447                       Microwave Massacre (1983)
9665                       Porky in Wackyland (1938)
8865                              The Lobster (2015)
9600                            Der Herr Karl (1961)
9300                             Kill Command (2016)
7028                              Powder Blue (2009)
6600                The Jane Austen Book Club (2007)
9479                           American Fable (2017)
9629                             Paddington 2 (2017)
7569                                Insidious (2010)
9354                        Vigilante Diaries (2016)
1707           

---------

## 1.1 - Movielens full - EDA

In [149]:
# read files and specify cols 

movies = "C:/Users/Noah7/code/Github/AI-Noah-Eliasson-OPA23/data/ml-latest-small/ml-latest-small/movies.csv"
ratings = "C:/Users/Noah7/code/Github/AI-Noah-Eliasson-OPA23/data/ml-latest-small/ml-latest-small/ratings.csv"

df_movies = pd.read_csv(movies, usecols=["movieId", "title"], dtype={"movieId": "int32", "title": "str"})
df_ratings= pd.read_csv(ratings, usecols=["userId", "movieId", "rating"], dtype={"userId": "int32", "movieId": "int32", "rating": "float32"})
df_ratings


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


### EDA

1.1 B)

In [261]:
df_merged = pd.merge(df_ratings, df_movies, on="movieId", how="inner")

title_count = df_merged["title"].value_counts()

top_10_rated = title_count[:10].reset_index()
top_10_rated.columns = ['title', 'count']


fig = px.bar(top_10_rated, x="title", y="count", title="The most rated movies")
fig.show()
fig.write_html("1_1B_plot")

1.1 C)

In [258]:

mean_ratings = df_merged.groupby("title")["rating"].mean()

mean_ratings = pd.DataFrame(mean_ratings).reset_index()
mean_ratings_10 = mean_ratings[mean_ratings["title"].isin(top_10_rated["title"])]
fig = px.bar(mean_ratings_10, x="title", y="rating", title="Mean of rating points")
fig.show()
fig.write_html("1_1C_plot")